# Searching and Downloading Data with SunPy

SunPy provides `Fido` a unified interface to many different solar physics data providers. As well as being a single user interface to search and download data in Python, `Fido` aims to be easy to extend to new data providers. These `Fido` plugins, normally referred to as clients, can be implemented in the `sunpy` package or any other package.

In this session we will go through how `Fido` works and how you can use it to query many different data sources at one and filter the results.

## A Simple Example

The most common place `Fido` will search is the [Virtual Solar Observatory (VSO)](https://sdac.virtualsolar.org/cgi/search) which provides a unified search API to many different data providers. Let's look for some HMI data through `Fido` and the VSO:

In [ ]:
from sunpy.net import Fido, attrs as a
import astropy.units as u

In [ ]:
Fido.search(a.Time("2020/12/10", "2020/12/11"))

In [ ]:
Fido.search(a.Time("2020/12/10", "2020/12/11"), a.Instrument.hmi)

In [ ]:
Fido.search(a.Time("2020/12/10", "2020/12/11"), a.Instrument.hmi, a.Sample(1*u.day))

In [ ]:
results = Fido.search(a.Time("2020/12/10", "2020/12/11"), a.Instrument.hmi, a.Sample(1*u.day), a.Physobs.los_magnetic_field)
results

In [ ]:
Fido.fetch(results)

## A more complex example: HMI and AIA

Let's extend this example and do some searches for both a HMI and some AIA data at the same time. To make this work we can combine attrs together using the `&` (and) and `|` (or) operators:

In [ ]:
time = a.Time("2020/11/10", "2020/11/11") & a.Sample(1*u.day)
time 

In [ ]:
hmi = a.Instrument.hmi & a.Physobs.los_magnetic_field
hmi

In [ ]:
aia = a.Instrument.aia & a.Wavelength(17.1*u.nm)
aia

In [ ]:
Fido.search(time, hmi | aia)

## Querying Multiple Sources

Both of these examples so far have focused on the VSO. Fido supports many different clients:

In [ ]:
Fido

### SUVI

In [ ]:
suvi = a.Instrument.suvi & a.Level.two

In [ ]:
Fido.search(a.Time("2020/11/10", "2020/11/10T00:00:10"),
            a.Wavelength(171*u.AA),
            a.Instrument.aia | suvi)

## JSOC

In addition to the VSO client which serves data via JSOC, sunpy has it's own [JSOC](http://jsoc.stanford.edu/ajax/exportdata.html) client. Which can be faster and more flexible than the VSO for large or complex exports. However, it is harder to use as the VSO has put a lot of effort into indexing the JSOC data in a way which is easier to search via common parameters like Instrument and physical type.

Let's start with a simple AIA search using the JSOC:

In [ ]:
a.jsoc.Series

In [ ]:
Fido.search(a.Time("2020/11/10", "2020/11/11"), a.Sample(1*u.hour), a.jsoc.Series.aia_lev1_euv_12s)

In [ ]:
Fido.search(a.Time("2020/11/10", "2020/11/11T06:00:00"),
            a.Sample(1*u.hour),
            a.jsoc.Series.aia_lev1_euv_12s | a.jsoc.Series.hmi_b_720s)

In [ ]:
aia = a.jsoc.Series.aia_lev1_euv_12s & a.Wavelength(304*u.AA) & a.jsoc.Segment.image
hmi = a.jsoc.Series.hmi_m_720s & a.jsoc.Segment.magnetogram

In [ ]:
results = Fido.search(a.Time("2020/11/10", "2020/11/10T00:50:00"),
                      a.Sample(1*u.hour),
                      aia | hmi,
                      a.jsoc.Notify("stuart@cadair.com"))

In [ ]:
results

In [ ]:
files = Fido.fetch(results)
files

### MapSequence

Map supports an iterable of disjoint maps called `MapSequence`, let's make one out of both these files we downloaded.

In [ ]:
%matplotlib widget

In [ ]:
import sunpy.map

In [ ]:
maps = sunpy.map.Map(files, sequence=True)

In [ ]:
maps

In [ ]:
maps[0].peek(cmap='hmimag')

## Querying the Heliophysics Events Knowlesgebase (HEK)

Sunpy has a client for querying the HEK, which can be used to search for flares and other kinds of events. See https://docs.sunpy.org/en/stable/guide/acquiring_data/hek.html for more details.

It's worth noting that in the upcoming sunpy 2.1 the HEK (and JSOC metadata) queries will be built into `Fido`, but the query syntax will remain largely the same.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

import sunpy.map

In [ ]:
from sunpy.net import hek

In [ ]:
client = hek.HEKClient()

In [ ]:
res = client.search(hek.attrs.Time("2020/12/01", "2020/12/16"), hek.attrs.EventType('FL'))

In [ ]:
res

This results object is a slightly customised, [astropy Table](https://docs.astropy.org/en/stable/table/index.html) which allows us to inspect it and work with the flare results. First we can list the keys.

In [ ]:
res.keys()

Let's have a look at the flare class:

In [ ]:
res['fl_goescls']

Let's select only the results which have a GOES flare class.

In [ ]:
flares = res[res['fl_goescls'].nonzero()]
flares

In [ ]:
flares['event_peaktime', 'fl_goescls', 'hpc_coord']

This hpc_coord isn't very useful, so let's convert it into a coordinate object we can use:

In [ ]:
point = flares['hpc_coord'][0]

In [ ]:
point

In [ ]:
point = point.strip("POINT()")
point

In [ ]:
point = point.split()
point

In [ ]:
list(map(float, point))

In [ ]:
coords = []
for c in flares['hpc_coord']:
    coords.append(list(map(float, c.strip("POINT()").split())))

Having cleaned up the coordinates, we can inject them back into the table, because you can put astropy coordinates into an astropy table.

In [ ]:
flares['hpc_coord'] = SkyCoord(coords, unit=u.arcsec, obstime=flares['event_peaktime'], observer="earth", frame="helioprojective")

In [ ]:
flares['event_peaktime', 'fl_goescls', 'hpc_coord']

Now let's create a simple map and plot the locations of all the flares on it.

In [ ]:
shape = (260,260)

In [ ]:
header = sunpy.map.make_fitswcs_header(shape,
                                       SkyCoord(0, 0, unit=u.arcsec, obstime=flares['event_peaktime'][0], observer="earth", frame="helioprojective"),
                                       scale=[10,10]*u.arcsec/u.pix)

In [ ]:
empty = sunpy.map.Map(np.zeros(shape)*np.nan, header)

In [ ]:
plt.figure()
ax = plt.subplot(projection=empty)
empty.plot(cmap="Blues")
empty.draw_grid(color='k')
ax.plot_coord(flares['hpc_coord'], "x")